In [ ]:
import pandas as pd
import numpy as np

from source.Files_operating import read_sht_df
from source.Plotting import plot_shoot
from source.Signal_processing import dbs_A_dFi, get_shoot_slices

In [ ]:
# Main params

F_ID = 41226  # NUM (ID) OF CURRENT SHOOT

# Paths
proj_path = "C:/Users/f.belous/Work/Projects/Plasma_analysis"  # GLOBAL PATH TO PROJECT DIR - EDIT ONES
sht_dir_path = proj_path + "/data/sht/all/"  # GLOBAL PATH TO SHOOTS SHT DIR - EDIT ONES
dbs_dir_path = proj_path + "/data/dbs/sht/"  # GLOBAL PATH TO Dref SHT FILES DIR - EDIT ONES

# Filter & smoothing params
w = 0.8  # recent filtering param (better don't touch)
smooth_length = 200  # DBS points ( smooth_length / 4 = timedelta (msc) )
plot_smoothing = 100  # Smoothing points on plot

In [ ]:
# Loading shoot data from SHT file in "sht_dir_path"

df = read_sht_df(f'sht{F_ID}', sht_dir_path,
                 column_names=[  #  SHT NAMES OF CHANNELS
                     "D-alfa  хорда R=50 cm",
                     "SXR 50 mkm",
                     "nl 42 cm (1.5мм) 64pi",
                     "МГД быстрый зонд верт.",
                     "МГД быстрый зонд рад.",
                 ],
                 data_names=[  # NAMES IN DF (eng & better to make it short & lowercase)
                    "d_alpha",
                     "sxr",
                     "nl",
                     "mgd_v",
                     "mgd_r",
                 ])

df["mhd"] = df.mgd_v.to_numpy() ** 2 + df.mgd_r.to_numpy() ** 2

df.describe()

In [ ]:
# Loading DBS data from "dbs_dir_path"
# Check existing of Dref file for this shoot!

channels = [3, 4, 5, 6, 7, 8]  # 1, 2, 3, 4, 5, 6, 7, 8 | DBS channels - to get frequency get journal of this shoot for DBS 
# (old: 1,2 - 39 GHz, ...; new: 1,2 - 20 GHz)
dbs_df = read_sht_df(f'Dref{F_ID}', dbs_dir_path,
                     column_names=[f"ch{channel}" for channel in channels])

dbs_df.describe()

In [ ]:
# Gets A & dFi
# Check filter param - "w" & smoothing param - "smooth_length"!
dbs_A_dFi(dbs_df, w, smooth_length)

dbs_df.describe()

In [ ]:
# Gets slices by SXR fall & split first slice (before first SXR fall)

res_slices_edges = get_shoot_slices(df.d_alpha, df.sxr)

---

In [ ]:
# Lists for saving data
# DO NOT RUN WHILE MARKING ONE SHOOT -- CLEAR ALL CURRENT DATA

df_groups_arr = []  # l_edge, r_edge, n, fr, fr_std
df_points_arr = []

In [ ]:
# Plotting cell
# You can set start index or input number of wanted slices (check twice)
start_index = 0
plot_shoot(F_ID, res_slices_edges, start_index, 2000,
           df, dbs_df, plot_smoothing, df_groups_arr, df_points_arr)

---
---

In [ ]:
# Stats of current data

print(f"Shot {F_ID}\n----")
print(f"Samples (total: {len(df_points_arr)}): EHO - {np.count_nonzero(np.array(df_points_arr)[:, 2] == 'eho')}, LCO - {np.count_nonzero(np.array(df_points_arr)[:, 2] == 'lco')}, dELM - {np.count_nonzero(np.array(df_points_arr)[:, 2] == 'delm')}")
print(f"Groups (total: {len(df_groups_arr)}): EHO - {np.count_nonzero(np.array(df_groups_arr)[:, -1] == 'eho')}, LCO - {np.count_nonzero(np.array(df_groups_arr)[:, -1] == 'lco')}, dELM - {np.count_nonzero(np.array(df_groups_arr)[:, -1] == 'delm')}")

In [ ]:
# SAVE CURRENT DATA

# l_edge, r_edge, n, fr, fr std
gr_df = pd.DataFrame(sorted(df_groups_arr, key=lambda x: x[0]), columns=["l_edge", "r_edge", "n", "fr", "fr_std", "mark"])
p_df = pd.DataFrame(sorted(df_points_arr, key=lambda x: x[0]), columns=["timepoint, ms", "fr, kHz", "mark"])

gr_df.to_csv(f"data/df/stats/groups/{F_ID}_groups_stats.csv", index=False)
p_df.to_csv(f"data/df/stats/samples/{F_ID}_points_dataset.csv", index=False)

# DO NOT FORGET TO CLEAR DATA FROM ARRAYS "df_groups_arr" "df_points_arr" BEFORE MARKING NEXT SHOOT

---